 # Reservas Airbnb 2023/2024

In [340]:
import pandas as pd
import re
from datetime import datetime
import glob
import numpy as np
from datetime import datetime
import calendar
from string import *
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm
from sklearn import preprocessing

In [341]:
def import_multiple_csv(column_sep=',',col_name_dic={},col_dtypes_dic={}, date_columns = []):
    """
    Imports all csv files from a folder and stacks them in one data frame.
    path: path to where files are located
    starts_with: beggining of files name
    column_sep: column separator
    col_name_dic: dictionary to rename columns
    col_dtypes_dic: dictionary to change data types
    
    Example: import_multiple_csv('C:/project/'
                                 ,'sales_report'
                                 ,column_sep=','
                                 ,col_name_dic={'Total Sales':'tot_sales'}
                                 ,col_dtypes_dic={'Total Sales':'int64'})
    """  
    #List with file names
    all_files = glob.glob("*.csv")
    #Empty list to store multiple Pandas Data Frames
    li = []
    #Import each file and append it to a list
    for filename in all_files:
        df = pd.read_csv(filename, sep = ',', parse_dates= ['Data de início','Data de término','Reservado'])
        li.append(df)

    # Concatenate all Data Frames from the list
    frame = pd.concat(li, axis=0, ignore_index=True)
    # Renaming columns
    frame.rename(col_name_dic, axis =1, inplace=True)
    return frame

In [342]:
# Importando todos os arquivos e concatenando todos em um só data frame (meio avançado)
df = import_multiple_csv(column_sep=',',col_name_dic={},col_dtypes_dic={}, date_columns = [])

/var/folders/3y/b_kj5_xd25n0pc7gt9_qym6r0000gn/T/ipykernel_1535/786814377.py:22: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df = pd.read_csv(filename, sep = ',', parse_dates= ['Data de início','Data de término','Reservado'])
/var/folders/3y/b_kj5_xd25n0pc7gt9_qym6r0000gn/T/ipykernel_1535/786814377.py:22: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df = pd.read_csv(filename, sep = ',', parse_dates= ['Data de início','Data de término','Reservado'])
/var/folders/3y/b_kj5_xd25n0pc7gt9_qym6r0000gn/T/ipykernel_1535/786814377.py:22: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df = pd.read_csv(filename, sep = ',', parse_dates= ['Data de iníci

In [343]:
# Convertendo para datetime
df['Data de início'] = pd.to_datetime(df['Data de início'], format = "%d/%m/%Y" )
df['Data de término'] = pd.to_datetime(df['Data de término'], format = "%d/%m/%Y" )

In [344]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 513 entries, 0 to 512
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Código de confirmação  513 non-null    object        
 1   Status                 513 non-null    object        
 2   Nome do hóspede        513 non-null    object        
 3   Entrar em contato      41 non-null     object        
 4   Nº de adultos          513 non-null    int64         
 5   Nº de crianças         513 non-null    int64         
 6   Nº de bebês            513 non-null    int64         
 7   Data de início         513 non-null    datetime64[ns]
 8   Data de término        513 non-null    datetime64[ns]
 9   Nº de noites           513 non-null    int64         
 10  Reservado              513 non-null    datetime64[ns]
 11  Anúncio                513 non-null    object        
 12  Ganhos                 513 non-null    object        
dtypes: da

In [345]:
df

,Código de confirmação,Status,Nome do hóspede,Entrar em contato,Nº de adultos,Nº de crianças,Nº de bebês,Data de início,Data de término,Nº de noites,Reservado,Anúncio,Ganhos
0,HMYM2TERN4,Hóspede anterior,Adilson Thiel,NaN,2,0,0,2023-08-12,2023-08-15,3,2023-08-07,Studio Praia Ipanema,"R$825,68"
1,HMK9A3HPXS,Hóspede anterior,Kevin K. Ozdamar,NaN,1,0,0,2023-08-11,2023-08-22,11,2023-08-08,Charmoso Apartamento no Leblon!,"R$4.167,14"
2,HMRSW4ZSKM,Hóspede anterior,Tati Medeiros,NaN,8,0,0,2023-08-11,2023-08-14,3,2023-07-11,Apê Amplo Ipanema 300 m da Praia,"R$1.744,39"
3,HMXPZ8WSSP,Hóspede anterior,Betina Lorscheitter,NaN,2,0,0,2023-08-11,2023-08-13,2,2023-07-11,Stúdio Praia Leblon,"R$697,75"
4,HMXP8HBHHK,Hóspede anterior,Ricardo Pinto,NaN,4,0,0,2023-08-10,2023-08-17,7,2023-08-02,Apê Amplo em Ipanema 300 m da Praia,"R$3.312,88"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
508,HM8358NW5F,Hóspede anterior,Alany Coutinho,NaN,6,0,0,2023-10-18,2023-10-22,4,2023-10-04,Super Apê com Jacuzzi Ipanema,"R$1.860,14"
509,HMRF2ZB5M5,Hóspede anterior,Daniel Faustino Da Silva,NaN,2,0,0,2023-10-16,2023-10-24,8,2023-09-03,Ipanema Studio 300 m da praia,"R$1.591,61"
510,HMEAYCM5Y5,Hóspede anterior,Macarena Cabello,NaN,8,0,0,2023-10-15,2023-10-19,4,2023-09-12,Apê Amplo em Ipanema 300 m da Praia,"R$1.944,02"
511,HMR82YSKQR,Hóspede anterior,Cinthia Brito,NaN,8,1,2,2023-10-13,2023-10-21,8,2023-09-03,Apê Amplo Ipanema 300 m da Praia,"R$6.876,80"


In [346]:
#criando coluna plataforma
df['Plataforma'] = 'Airbnb'

In [347]:
df[2:3]

,Código de confirmação,Status,Nome do hóspede,Entrar em contato,Nº de adultos,Nº de crianças,Nº de bebês,Data de início,Data de término,Nº de noites,Reservado,Anúncio,Ganhos,Plataforma
2,HMRSW4ZSKM,Hóspede anterior,Tati Medeiros,NaN,8,0,0,2023-08-11,2023-08-14,3,2023-07-11,Apê Amplo Ipanema 300 m da Praia,"R$1.744,39",Airbnb


In [348]:
# importando base da Stays
df2 = pd.read_excel('/Users/ruynunes/Documents/Novos_Projetos_Python/Aluguel_Temporada_Análise_de_Reservas/2023_05_01_2024_01_30_propX_ownerX.xlsx',
                   parse_dates = ['Chegada'])

/var/folders/3y/b_kj5_xd25n0pc7gt9_qym6r0000gn/T/ipykernel_1535/4203025774.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df2 = pd.read_excel('/Users/ruynunes/Documents/Novos_Projetos_Python/Aluguel_Temporada_Análise_de_Reservas/2023_05_01_2024_01_30_propX_ownerX.xlsx',


In [349]:
df2['Chegada'] = df2['Chegada'].str.replace('jan', 'January')
df2['Chegada'] = df2['Chegada'].str.replace('fev', 'February')
df2['Chegada'] = df2['Chegada'].str.replace('mar', 'March')
df2['Chegada'] = df2['Chegada'].str.replace('abr', 'April')
df2['Chegada'] = df2['Chegada'].str.replace('mai', 'May')
df2['Chegada'] = df2['Chegada'].str.replace('jun', 'June')
df2['Chegada'] = df2['Chegada'].str.replace('jul', 'July')
df2['Chegada'] = df2['Chegada'].str.replace('ago', 'August')
df2['Chegada'] = df2['Chegada'].str.replace('set', 'September')
df2['Chegada'] = df2['Chegada'].str.replace('out', 'October')
df2['Chegada'] = df2['Chegada'].str.replace('nov', 'November')
df2['Chegada'] = df2['Chegada'].str.replace('dez', 'December')
df2['Data de checkout'] = df2['Data de checkout'].str.replace('jan', 'January')
df2['Data de checkout'] = df2['Data de checkout'].str.replace('fev', 'February')
df2['Data de checkout'] = df2['Data de checkout'].str.replace('mar', 'March')
df2['Data de checkout'] = df2['Data de checkout'].str.replace('abr', 'April')
df2['Data de checkout'] = df2['Data de checkout'].str.replace('mai', 'May')
df2['Data de checkout'] = df2['Data de checkout'].str.replace('jun', 'June')
df2['Data de checkout'] = df2['Data de checkout'].str.replace('jul', 'July')
df2['Data de checkout'] = df2['Data de checkout'].str.replace('ago', 'August')
df2['Data de checkout'] = df2['Data de checkout'].str.replace('set', 'September')
df2['Data de checkout'] = df2['Data de checkout'].str.replace('out', 'October')
df2['Data de checkout'] = df2['Data de checkout'].str.replace('nov', 'November')
df2['Data de checkout'] = df2['Data de checkout'].str.replace('dez', 'December')
df2['Data de Criação'] = df2['Data de Criação'].str.replace('jan', 'January')
df2['Data de Criação'] = df2['Data de Criação'].str.replace('fev', 'February')
df2['Data de Criação'] = df2['Data de Criação'].str.replace('mar', 'March')
df2['Data de Criação'] = df2['Data de Criação'].str.replace('abr', 'April')
df2['Data de Criação'] = df2['Data de Criação'].str.replace('mai', 'May')
df2['Data de Criação'] = df2['Data de Criação'].str.replace('jun', 'June')
df2['Data de Criação'] = df2['Data de Criação'].str.replace('jul', 'July')
df2['Data de Criação'] = df2['Data de Criação'].str.replace('ago', 'August')
df2['Data de Criação'] = df2['Data de Criação'].str.replace('set', 'September')
df2['Data de Criação'] = df2['Data de Criação'].str.replace('out', 'October')
df2['Data de Criação'] = df2['Data de Criação'].str.replace('nov', 'November')
df2['Data de Criação'] = df2['Data de Criação'].str.replace('dez', 'December')

In [350]:
pd.set_option('display.max_columns', 500)

In [351]:
df2.head()

,Nº,Mês,Chegada,Data de checkout,ID do Anúncio,Nome Interno do Anúncio,Reserva,Nome do Hóspede,Número de telefone,Total de Hóspedes,Tipo de documento,Número de documento,OTA's external code,TC (Terms / Conditions),Canal,Data de Criação,Data de Repasse,Número de Noites,Moeda,Valor por Noite,Total da Reserva,Base de Cálculo do Repasse,Preço de Venda Corrigido,Comissão da Empresa,Preço de Compra,Taxas: Taxa de Limpeza,Taxa do Proprietário: API Decolar,Taxa do Proprietário: API airbnb,Taxa do Proprietário: API booking.com,Taxa do Proprietário: Airbnb,Taxas Adicionais: ISS,Total de Taxas de Repasse,Total da Fatura de Hospedagem,Total de Serviços Extras,Primeiro Nome do Hóspede,Sobrenome do Hóspede,E-mail,Status da Reserva,Observação,Tem Avaliação do Hóspede?,Original Review Rating,Converted Review Rating,Agente,Cancel date,Adults Count,Children Count,Infants Count,Listing Internal ID,Guest Internal ID
0,1,janeiro,29 January 2024,07 February 2024,XD03H,501,DU02I,Jorge Aburto,+56 9 4029 0634,7,NaN,NaN,HMFHKQMHZ3,NaN,API airbnb,18 January 2024,NaN,9,BRL,725.56,6830.04,6830.04,5774.93,0,5774.93,300.0,NaN,1055.11,NaN,NaN,NaN,1055.11,6830.04,0,Jorge,Aburto,NaN,reserva,NaN,NaN,NaN,NaN,Ruy Nunes,NaN,4,3,0,64d3c1f3e824ab225c37160e,65a9726bbb8acd83ebe872b8
1,2,janeiro,29 January 2024,02 February 2024,XF01H,Gomes 2,DU01I,Javiera Hernandez Pinilla,+56 9 3066 2527,2,NaN,NaN,HMDDSXJJ5F,NaN,API airbnb,03 January 2024,NaN,4,BRL,184.00,916.00,916.00,774.49,0,774.49,180.0,NaN,141.51,NaN,NaN,NaN,141.51,916.00,0,Javiera,Hernandez Pinilla,NaN,reserva,NaN,NaN,NaN,NaN,Ruy Nunes,NaN,2,0,0,64d6408812d4af9a23f6e713,659615049425649405a9e64f
2,3,janeiro,25 January 2024,01 February 2024,XF03H,General,DQ03I,Thalita Meinberg,+55 31 97560 4406,6,NaN,NaN,HMKX38PFS4,NaN,API airbnb,06 January 2024,NaN,7,BRL,656.39,4794.72,4794.72,4054.03,0,4054.03,200.0,NaN,740.69,NaN,NaN,NaN,740.69,4794.72,0,Thalita,Meinberg,NaN,reserva,NaN,NaN,NaN,NaN,Ruy Nunes,NaN,6,0,0,64d6429212d4af9a23f70783,6599c19d9139451644d90f48
3,4,janeiro,25 January 2024,30 January 2024,XF10H,afrânio,DQ01I,Camila Secco,+55 19 99188 5533,4,NaN,NaN,HMPDX8R2DE,NaN,API airbnb,12 December 2023,NaN,5,BRL,395.00,2175.00,2175.00,1839.01,0,1839.01,200.0,NaN,335.99,NaN,NaN,NaN,335.99,2175.00,0,Camila,Secco,NaN,reserva,NaN,NaN,NaN,NaN,Ruy Nunes,NaN,4,0,0,64d64c6a12d4af9a23f7ba1e,65789851d4be6f50a6211f14
4,5,janeiro,24 January 2024,29 January 2024,XF01H,Gomes 2,DP03I,Stefany Oliveira,+55 11 95274 2927,2,NaN,NaN,HMKW5BXYXW,NaN,API airbnb,21 January 2024,NaN,5,BRL,248.40,1422.00,1422.00,1202.33,0,1202.33,180.0,NaN,219.67,NaN,NaN,NaN,219.67,1422.00,0,Stefany,Oliveira,NaN,reserva,NaN,NaN,NaN,NaN,Ruy Nunes,NaN,2,0,0,64d6408812d4af9a23f6e713,65ad3ae6037dd6ac5512050f


In [352]:
#convertendo para datetime
df2['Chegada'] = pd.to_datetime(df2['Chegada'])
df2['Data de checkout'] = pd.to_datetime(df2['Data de checkout'])
df2['Data de Criação'] = pd.to_datetime(df2['Data de Criação'])

In [353]:
#organizando colunas da base stays
dfstays = df2[["OTA's external code", 'Chegada','Data de checkout', 'Nome Interno do Anúncio',
               'Total de Hóspedes','Adults Count', 'Children Count', 'Infants Count', 'Canal','Data de Criação', 'Número de Noites', 'Valor por Noite', 'Total da Reserva','Preço de Venda Corrigido',
              'Status da Reserva', ]]

# Continuação análise de temporada.

-Arrancar os dias, meses e anos. 

- Padronizar canal
- Padronizar nomes de colunas
- Calcular comissão Airbnb
- Padronizar status da reserva
- Padronizar nome do anúncio (da acomodação)
- Criar coluna bairro
- Criar coluna calculando número de dias entre a reserva e a chegada
- Criar coluna preço por hospede
- Criar valor por noite na tabela do Airbnb
- Criar coluna tipo de acomodação (quarto e apto inteiro)


In [354]:
dfstays.head(4)

,OTA's external code,Chegada,Data de checkout,Nome Interno do Anúncio,Total de Hóspedes,Adults Count,Children Count,Infants Count,Canal,Data de Criação,Número de Noites,Valor por Noite,Total da Reserva,Preço de Venda Corrigido,Status da Reserva
0,HMFHKQMHZ3,2024-01-29,2024-02-07,501,7,4,3,0,API airbnb,2024-01-18,9,725.56,6830.04,5774.93,reserva
1,HMDDSXJJ5F,2024-01-29,2024-02-02,Gomes 2,2,2,0,0,API airbnb,2024-01-03,4,184.00,916.00,774.49,reserva
2,HMKX38PFS4,2024-01-25,2024-02-01,General,6,6,0,0,API airbnb,2024-01-06,7,656.39,4794.72,4054.03,reserva
3,HMPDX8R2DE,2024-01-25,2024-01-30,afrânio,4,4,0,0,API airbnb,2023-12-12,5,395.00,2175.00,1839.01,reserva


In [355]:
df.head()

,Código de confirmação,Status,Nome do hóspede,Entrar em contato,Nº de adultos,Nº de crianças,Nº de bebês,Data de início,Data de término,Nº de noites,Reservado,Anúncio,Ganhos,Plataforma
0,HMYM2TERN4,Hóspede anterior,Adilson Thiel,NaN,2,0,0,2023-08-12,2023-08-15,3,2023-08-07,Studio Praia Ipanema,"R$825,68",Airbnb
1,HMK9A3HPXS,Hóspede anterior,Kevin K. Ozdamar,NaN,1,0,0,2023-08-11,2023-08-22,11,2023-08-08,Charmoso Apartamento no Leblon!,"R$4.167,14",Airbnb
2,HMRSW4ZSKM,Hóspede anterior,Tati Medeiros,NaN,8,0,0,2023-08-11,2023-08-14,3,2023-07-11,Apê Amplo Ipanema 300 m da Praia,"R$1.744,39",Airbnb
3,HMXPZ8WSSP,Hóspede anterior,Betina Lorscheitter,NaN,2,0,0,2023-08-11,2023-08-13,2,2023-07-11,Stúdio Praia Leblon,"R$697,75",Airbnb
4,HMXP8HBHHK,Hóspede anterior,Ricardo Pinto,NaN,4,0,0,2023-08-10,2023-08-17,7,2023-08-02,Apê Amplo em Ipanema 300 m da Praia,"R$3.312,88",Airbnb


In [356]:
#Criando novas colunas e organizando a base airbnb
df = df[['Anúncio','Plataforma', 'Reservado','Data de início', 'Data de término','Nº de noites','Nº de adultos','Nº de crianças','Nº de bebês', 'Ganhos', 'Status', 'Código de confirmação']]
df['Atendecência Reserva (dias)']= 0
df['Total da Reserva'] = 0
df['Preço de Venda Corrigido'] = 0
df['Valor por Noite'] = 0
df['Total de Hóspedes'] = 0

df = df[['Anúncio', 'Plataforma', 'Reservado','Atendecência Reserva (dias)', 'Data de início', 'Data de término','Nº de noites','Nº de adultos','Nº de crianças','Nº de bebês','Total de Hóspedes', 'Total da Reserva', 'Preço de Venda Corrigido', 'Valor por Noite', 'Ganhos', 'Status', 'Código de confirmação']]


/var/folders/3y/b_kj5_xd25n0pc7gt9_qym6r0000gn/T/ipykernel_1535/1458508029.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Atendecência Reserva (dias)']= 0


In [357]:
dfstays.columns

Index(['OTA's external code', 'Chegada', 'Data de checkout',
       'Nome Interno do Anúncio', 'Total de Hóspedes', 'Adults Count',
       'Children Count', 'Infants Count', 'Canal', 'Data de Criação',
       'Número de Noites', 'Valor por Noite', 'Total da Reserva',
       'Preço de Venda Corrigido', 'Status da Reserva'],
      dtype='object')

In [358]:
df.head()

,Anúncio,Plataforma,Reservado,Atendecência Reserva (dias),Data de início,Data de término,Nº de noites,Nº de adultos,Nº de crianças,Nº de bebês,Total de Hóspedes,Total da Reserva,Preço de Venda Corrigido,Valor por Noite,Ganhos,Status,Código de confirmação
0,Studio Praia Ipanema,Airbnb,2023-08-07,0,2023-08-12,2023-08-15,3,2,0,0,0,0,0,0,"R$825,68",Hóspede anterior,HMYM2TERN4
1,Charmoso Apartamento no Leblon!,Airbnb,2023-08-08,0,2023-08-11,2023-08-22,11,1,0,0,0,0,0,0,"R$4.167,14",Hóspede anterior,HMK9A3HPXS
2,Apê Amplo Ipanema 300 m da Praia,Airbnb,2023-07-11,0,2023-08-11,2023-08-14,3,8,0,0,0,0,0,0,"R$1.744,39",Hóspede anterior,HMRSW4ZSKM
3,Stúdio Praia Leblon,Airbnb,2023-07-11,0,2023-08-11,2023-08-13,2,2,0,0,0,0,0,0,"R$697,75",Hóspede anterior,HMXPZ8WSSP
4,Apê Amplo em Ipanema 300 m da Praia,Airbnb,2023-08-02,0,2023-08-10,2023-08-17,7,4,0,0,0,0,0,0,"R$3.312,88",Hóspede anterior,HMXP8HBHHK


In [359]:
#padronizando os nomes de colunas da base airbnb em relação a base stays
dicty = {'Código de confirmação': "OTA's external code", 'Data de início': 'Chegada',
        'Data de término': 'Data de checkout',
       'Anúncio':'Nome Interno do Anúncio', 'Nº de adultos': 'Adults Count',
       'Nº de crianças': 'Children Count', 'Nº de bebês': 'Infants Count', 'Plataforma':'Canal', 'Reservado': 'Data de Criação',
       'Nº de noites': 'Número de Noites', 'Status': 'Status da Reserva'}

df.rename(columns=dicty,inplace=True)
df['Total de Hóspedes'] = df['Adults Count'] + df['Children Count'] + df['Infants Count']




In [360]:
df.head()

,Nome Interno do Anúncio,Canal,Data de Criação,Atendecência Reserva (dias),Chegada,Data de checkout,Número de Noites,Adults Count,Children Count,Infants Count,Total de Hóspedes,Total da Reserva,Preço de Venda Corrigido,Valor por Noite,Ganhos,Status da Reserva,OTA's external code
0,Studio Praia Ipanema,Airbnb,2023-08-07,0,2023-08-12,2023-08-15,3,2,0,0,2,0,0,0,"R$825,68",Hóspede anterior,HMYM2TERN4
1,Charmoso Apartamento no Leblon!,Airbnb,2023-08-08,0,2023-08-11,2023-08-22,11,1,0,0,1,0,0,0,"R$4.167,14",Hóspede anterior,HMK9A3HPXS
2,Apê Amplo Ipanema 300 m da Praia,Airbnb,2023-07-11,0,2023-08-11,2023-08-14,3,8,0,0,8,0,0,0,"R$1.744,39",Hóspede anterior,HMRSW4ZSKM
3,Stúdio Praia Leblon,Airbnb,2023-07-11,0,2023-08-11,2023-08-13,2,2,0,0,2,0,0,0,"R$697,75",Hóspede anterior,HMXPZ8WSSP
4,Apê Amplo em Ipanema 300 m da Praia,Airbnb,2023-08-02,0,2023-08-10,2023-08-17,7,4,0,0,4,0,0,0,"R$3.312,88",Hóspede anterior,HMXP8HBHHK


In [361]:
dfstays.columns

Index(['OTA's external code', 'Chegada', 'Data de checkout',
       'Nome Interno do Anúncio', 'Total de Hóspedes', 'Adults Count',
       'Children Count', 'Infants Count', 'Canal', 'Data de Criação',
       'Número de Noites', 'Valor por Noite', 'Total da Reserva',
       'Preço de Venda Corrigido', 'Status da Reserva'],
      dtype='object')

In [362]:
#Criando colunas que já existe na base airbnb na base stays 
dfstays["Atendecência Reserva (dias)"] = 0
dfstays['Ganhos'] = 0
dfstays= dfstays[['Nome Interno do Anúncio', 'Canal' , 'Data de Criação', 'Atendecência Reserva (dias)', 'Chegada', 
                 'Data de checkout','Número de Noites', 'Adults Count', 'Children Count', 'Infants Count',
                 'Total de Hóspedes', 'Total da Reserva', 'Preço de Venda Corrigido', 'Valor por Noite', 'Ganhos', 
                 'Status da Reserva', "OTA's external code"]]
dfstays.head()

/var/folders/3y/b_kj5_xd25n0pc7gt9_qym6r0000gn/T/ipykernel_1535/534979160.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfstays["Atendecência Reserva (dias)"] = 0
/var/folders/3y/b_kj5_xd25n0pc7gt9_qym6r0000gn/T/ipykernel_1535/534979160.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfstays['Ganhos'] = 0


,Nome Interno do Anúncio,Canal,Data de Criação,Atendecência Reserva (dias),Chegada,Data de checkout,Número de Noites,Adults Count,Children Count,Infants Count,Total de Hóspedes,Total da Reserva,Preço de Venda Corrigido,Valor por Noite,Ganhos,Status da Reserva,OTA's external code
0,501,API airbnb,2024-01-18,0,2024-01-29,2024-02-07,9,4,3,0,7,6830.04,5774.93,725.56,0,reserva,HMFHKQMHZ3
1,Gomes 2,API airbnb,2024-01-03,0,2024-01-29,2024-02-02,4,2,0,0,2,916.00,774.49,184.00,0,reserva,HMDDSXJJ5F
2,General,API airbnb,2024-01-06,0,2024-01-25,2024-02-01,7,6,0,0,6,4794.72,4054.03,656.39,0,reserva,HMKX38PFS4
3,afrânio,API airbnb,2023-12-12,0,2024-01-25,2024-01-30,5,4,0,0,4,2175.00,1839.01,395.00,0,reserva,HMPDX8R2DE
4,Gomes 2,API airbnb,2024-01-21,0,2024-01-24,2024-01-29,5,2,0,0,2,1422.00,1202.33,248.40,0,reserva,HMKW5BXYXW


In [363]:
df.columns

Index(['Nome Interno do Anúncio', 'Canal', 'Data de Criação',
       'Atendecência Reserva (dias)', 'Chegada', 'Data de checkout',
       'Número de Noites', 'Adults Count', 'Children Count', 'Infants Count',
       'Total de Hóspedes', 'Total da Reserva', 'Preço de Venda Corrigido',
       'Valor por Noite', 'Ganhos', 'Status da Reserva',
       'OTA's external code'],
      dtype='object')

In [364]:
df2 = df

In [365]:
#df2 = df2.merge(dfstays[[ "OTA's external code"]], 
                              #on="OTA's external code", how='right', validate='m:1')

In [366]:
#concatenando as 2 bases
df2 = pd.concat([df2, dfstays])

In [367]:
df2

,Nome Interno do Anúncio,Canal,Data de Criação,Atendecência Reserva (dias),Chegada,Data de checkout,Número de Noites,Adults Count,Children Count,Infants Count,Total de Hóspedes,Total da Reserva,Preço de Venda Corrigido,Valor por Noite,Ganhos,Status da Reserva,OTA's external code
0,Studio Praia Ipanema,Airbnb,2023-08-07,0,2023-08-12,2023-08-15,3,2,0,0,2,0.0,0.00,0.0,"R$825,68",Hóspede anterior,HMYM2TERN4
1,Charmoso Apartamento no Leblon!,Airbnb,2023-08-08,0,2023-08-11,2023-08-22,11,1,0,0,1,0.0,0.00,0.0,"R$4.167,14",Hóspede anterior,HMK9A3HPXS
2,Apê Amplo Ipanema 300 m da Praia,Airbnb,2023-07-11,0,2023-08-11,2023-08-14,3,8,0,0,8,0.0,0.00,0.0,"R$1.744,39",Hóspede anterior,HMRSW4ZSKM
3,Stúdio Praia Leblon,Airbnb,2023-07-11,0,2023-08-11,2023-08-13,2,2,0,0,2,0.0,0.00,0.0,"R$697,75",Hóspede anterior,HMXPZ8WSSP
4,Apê Amplo em Ipanema 300 m da Praia,Airbnb,2023-08-02,0,2023-08-10,2023-08-17,7,4,0,0,4,0.0,0.00,0.0,"R$3.312,88",Hóspede anterior,HMXP8HBHHK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,afrânio,API airbnb,2023-08-15,0,2023-05-26,2023-05-29,3,2,0,0,2,1010.0,978.80,270.0,0,reserva,HME5HRWZQZ
258,Ataulfo 1,API airbnb,2023-08-15,0,2023-05-25,2023-05-29,4,2,0,0,2,1610.0,1560.26,190.0,0,reserva,HMHHM9AD4Y
259,afrânio,API airbnb,2023-08-15,0,2023-05-21,2023-05-26,5,1,0,0,1,1450.0,1405.21,250.0,0,reserva,HMZ8825SQC
260,Ataulfo 1,API airbnb,2023-08-15,0,2023-05-14,2023-05-23,9,2,0,0,2,2870.8,2782.09,141.2,0,reserva,HM8XCNT9WS


In [368]:
#retirando linhas duplicadas e mantendo as linhas da base airbnb
df2 = df2.drop_duplicates(subset="OTA's external code", keep="first", inplace=False)

In [369]:
df2['Canal'].unique()

array(['Airbnb', 'API booking.com', 'API expedia', 'API Decolar'],
      dtype=object)

In [370]:
df2[ (df2['Canal'] == 'API expedia')]

,Nome Interno do Anúncio,Canal,Data de Criação,Atendecência Reserva (dias),Chegada,Data de checkout,Número de Noites,Adults Count,Children Count,Infants Count,Total de Hóspedes,Total da Reserva,Preço de Venda Corrigido,Valor por Noite,Ganhos,Status da Reserva,OTA's external code
15,Gomes 2,API expedia,2023-10-09,0,2024-01-15,2024-01-17,2,2,0,0,2,375.76,375.76,187.88,0,reserva,101734428
36,Gomes 2,API expedia,2023-10-19,0,2023-12-29,2024-01-03,5,3,0,0,3,1148.10,1148.10,229.62,0,reserva,107853324
156,Gomes 2,API expedia,2023-09-12,0,2023-09-15,2023-09-18,3,1,0,0,1,563.64,563.64,187.88,0,reserva,85720549


In [371]:
df2['Tipo de Acomodação'] = 0

/var/folders/3y/b_kj5_xd25n0pc7gt9_qym6r0000gn/T/ipykernel_1535/3528067657.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Tipo de Acomodação'] = 0


In [372]:
df2.head(1)

,Nome Interno do Anúncio,Canal,Data de Criação,Atendecência Reserva (dias),Chegada,Data de checkout,Número de Noites,Adults Count,Children Count,Infants Count,Total de Hóspedes,Total da Reserva,Preço de Venda Corrigido,Valor por Noite,Ganhos,Status da Reserva,OTA's external code,Tipo de Acomodação
0,Studio Praia Ipanema,Airbnb,2023-08-07,0,2023-08-12,2023-08-15,3,2,0,0,2,0.0,0.0,0.0,"R$825,68",Hóspede anterior,HMYM2TERN4,0


In [373]:
df2['Nome Interno do Anúncio'].unique()

array(['Studio Praia Ipanema', 'Charmoso Apartamento no Leblon!',
       'Apê Amplo Ipanema 300 m da Praia', 'Stúdio Praia Leblon',
       'Apê Amplo em Ipanema 300 m da Praia',
       'Ipanema Studio 300 m da praia', 'Super Apê com Jacuzzi Ipanema',
       'Apartamento Leblon Vista Mar!',
       'Copanema linda kitnet\n300 m da praia e Arpoador',
       'Apê Ipanema à 600m da praia', 'Ipanema à 200 m da praia',
       'Ipanema à 300 m da Praia',
       'Casa Cercada pela Natureza em Visconde de Mauá',
       'Ipanema à 350 m da Praia', 'Quarto solteiro Ipanema',
       'Suíte em Ipanema 300m da praia',
       '300 metros da praia em Ipanema.', 'Ocean View',
       'Leblon Vista Mar', 'Ipanema Amplo Vista Mar',
       'Praia Stúdio Ipanema',
       'Super Suíte em Ipanema com Hidromassagem!', 'Francisco',
       'Gomes 2', 'Ataulfo 1', 'afrânio', 'Ataulfo 2', 'Nascimento', 501],
      dtype=object)

In [374]:
#Padronizando os nomes dos anúncios
df2.loc[(df2['Nome Interno do Anúncio'].str.contains(pat = 'General 2', na = False))
       , 'Nome Interno do Anúncio'] = 'Super Apê com Jacuzzi Ipanema!'
df2.loc[(df2['Nome Interno do Anúncio'].str.contains(pat = 'Nascimento', na = False))
       , 'Nome Interno do Anúncio'] = 'Ipanema Amplo Vista Mar!'
df2.loc[(df2['Nome Interno do Anúncio'].str.contains(pat = 'Ocean', na = False))
       , 'Nome Interno do Anúncio'] = 'Ipanema Amplo Vista Mar!'
df2.loc[(df2['Nome Interno do Anúncio'].str.contains(pat = 'Gomes 1', na = False))
       , 'Nome Interno do Anúncio'] = 'Apê Amplo em Ipanema 300 m da Praia'
df2.loc[(df2['Nome Interno do Anúncio'].str.contains(pat = 'Gomes 2', na = False))
       , 'Nome Interno do Anúncio'] = 'Studio Praia Ipanema'
df2.loc[(df2['Nome Interno do Anúncio'].str.contains(pat = 'Francisco', na = False))
       , 'Nome Interno do Anúncio'] = 'Ipanema Studio 300 m da praia'
df2.loc[(df2['Nome Interno do Anúncio'].str.contains(pat = 'General', na = False))
       , 'Nome Interno do Anúncio'] = 'Super Apê com Jacuzzi Ipanema'
df2.loc[(df2['Nome Interno do Anúncio'].str.contains(pat = 'Ataulfo 2', na = False))
       , 'Nome Interno do Anúncio'] = 'Charmoso Apartamento no Leblon'
df2.loc[(df2['Nome Interno do Anúncio'].str.contains(pat = 'afrânio', na = False))
       , 'Nome Interno do Anúncio'] = 'Stúdio Praia Leblon'
df2.loc[(df2['Nome Interno do Anúncio'].str.contains(pat = 'Super Suíte em Ipanema com Hidromassagem!', na = False))
       , 'Nome Interno do Anúncio'] = 'Super Apê com Jacuzzi Ipanema'
df2.loc[(df2['Nome Interno do Anúncio'].str.contains(pat = 'Leblon Vista Mar', na = False))
       , 'Nome Interno do Anúncio'] = 'Apartamento Leblon Vista Mar!'
df2.loc[(df2['Nome Interno do Anúncio'].str.contains(pat = 'Ataulfo 1', na = False))
       , 'Nome Interno do Anúncio'] = 'Apartamento Leblon Vista Mar!'

In [375]:
#criando a coluna bairro e preenchendo seus campos
df2['Bairro'] = 'hello'
df2.loc[(df2['Nome Interno do Anúncio'].str.contains(pat = '501', na = False))
       , 'Nome Interno do Anúncio'] = 'Apê Amplo Ipanema 300 m da Praia'

df2.loc[(df2['Nome Interno do Anúncio'].str.contains(pat = 'Ipanema', na = False))
       , 'Bairro'] = 'Ipanema'
df2.loc[(df2['Nome Interno do Anúncio'].str.contains(pat = 'ipanema', na = False))
       , 'Bairro'] = 'Ipanema'
df2.loc[(df2['Nome Interno do Anúncio'].str.contains(pat = 'Leblon', na = False))
       , 'Bairro'] = 'Leblon'
df2.loc[(df2['Nome Interno do Anúncio'].str.contains(pat = 'leblon', na = False))
       , 'Bairro'] = 'Leblon'
df2.loc[(df2['Nome Interno do Anúncio'].str.contains(pat = 'Ocean', na = False))
       , 'Bairro'] = 'Ipanema'
df2.loc[(df2['Nome Interno do Anúncio'].str.contains(pat = 'leblon', na = False))
       , 'Bairro'] = 'Ipanema'
df2.loc[(df2['Nome Interno do Anúncio'].str.contains(pat = 'Copanema', na = False))
       , 'Bairro'] = 'Ipanema'
df2.loc[(df2['Nome Interno do Anúncio'].str.contains(pat = 'Natureza', na = False))
       , 'Bairro'] = 'Visconde de Mauá'





df2.tail()

/var/folders/3y/b_kj5_xd25n0pc7gt9_qym6r0000gn/T/ipykernel_1535/473351017.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Bairro'] = 'hello'


,Nome Interno do Anúncio,Canal,Data de Criação,Atendecência Reserva (dias),Chegada,Data de checkout,Número de Noites,Adults Count,Children Count,Infants Count,Total de Hóspedes,Total da Reserva,Preço de Venda Corrigido,Valor por Noite,Ganhos,Status da Reserva,OTA's external code,Tipo de Acomodação,Bairro
111,Apartamento Leblon Vista Mar!,API Decolar,2023-10-04,0,2023-10-24,2023-10-29,5,1,0,0,1,265.01,225.26,0.86,0,reserva,23513591002,0,Leblon
114,Stúdio Praia Leblon,API Decolar,2023-10-05,0,2023-10-22,2023-10-30,8,2,0,0,2,3264.00,2774.40,366.49,0,reserva,23527046602,0,Leblon
117,Charmoso Apartamento no Leblon,API Decolar,2023-10-09,0,2023-10-19,2023-10-22,3,1,0,0,1,1420.01,1207.01,387.51,0,reserva,23576110602,0,Leblon
146,501,Airbnb,2023-09-02,0,2023-09-23,2023-09-30,7,7,0,0,7,4000.00,4000.00,571.43,0,reserva,NaN,0,hello
156,Studio Praia Ipanema,API expedia,2023-09-12,0,2023-09-15,2023-09-18,3,1,0,0,1,563.64,563.64,187.88,0,reserva,85720549,0,Ipanema


In [376]:
df2.loc[(df2['Nome Interno do Anúncio'].str.contains(pat = '501', na = False))
       , 'Nome Interno do Anúncio'] = 'Apê Amplo Ipanema 300 m da Praia'

In [377]:
# Criando a coluna comissão plataforma
df2['Comissão Plataforma'] = df2['Total da Reserva'] - df2['Preço de Venda Corrigido']
df2.tail(2)

/var/folders/3y/b_kj5_xd25n0pc7gt9_qym6r0000gn/T/ipykernel_1535/530760181.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Comissão Plataforma'] = df2['Total da Reserva'] - df2['Preço de Venda Corrigido']


,Nome Interno do Anúncio,Canal,Data de Criação,Atendecência Reserva (dias),Chegada,Data de checkout,Número de Noites,Adults Count,Children Count,Infants Count,Total de Hóspedes,Total da Reserva,Preço de Venda Corrigido,Valor por Noite,Ganhos,Status da Reserva,OTA's external code,Tipo de Acomodação,Bairro,Comissão Plataforma
146,501,Airbnb,2023-09-02,0,2023-09-23,2023-09-30,7,7,0,0,7,4000.00,4000.00,571.43,0,reserva,NaN,0,hello,0.0
156,Studio Praia Ipanema,API expedia,2023-09-12,0,2023-09-15,2023-09-18,3,1,0,0,1,563.64,563.64,187.88,0,reserva,85720549,0,Ipanema,0.0


In [378]:
#Padronizando os canais de hospedagem
df2.loc[(df2['Canal'] == 'API booking.com' )
       , 'Canal'] = 'Booking'
df2.loc[(df2['Canal'] == 'API expedia' )
       , 'Canal'] = 'Expedia'
df2.loc[(df2['Canal'] == 'API Decolar' )
       , 'Canal'] = 'Decolar'

In [379]:
#Transformando a coluna ganho em float
df2.loc[(df2['Ganhos'] == 0), 'Ganhos'] = '0'
df2['Ganhos'] = df2['Ganhos'].map(lambda x: x.lstrip('R$'))
df2['Ganhos'] = df2['Ganhos'].str.replace('-R$', '-')                                  
df2['Ganhos'] = df2['Ganhos'].str.replace('.', '')
df2['Ganhos'] = df2['Ganhos'].str.replace(',', '.')
df2['Ganhos'] = df2['Ganhos'].astype(float)

/var/folders/3y/b_kj5_xd25n0pc7gt9_qym6r0000gn/T/ipykernel_1535/2917812620.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Ganhos'] = df2['Ganhos'].map(lambda x: x.lstrip('R$'))
/var/folders/3y/b_kj5_xd25n0pc7gt9_qym6r0000gn/T/ipykernel_1535/2917812620.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Ganhos'] = df2['Ganhos'].str.replace('-R$', '-')
/var/folders/3y/b_kj5_xd25n0pc7gt9_qym6r0000gn/T/ipykernel_1535/2917812620.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy 

In [380]:
#Arrumando os valores de total reserva e comissão plataforma
df2 = df2.reset_index(drop=True)
df2.loc[(df2['Canal'] == 'Airbnb' )
       , 'Total da Reserva'] = df2['Ganhos'].apply(lambda x: x*1.0/0.85)
df2.loc[(df2['Canal'] == 'Airbnb' )
       , 'Comissão Plataforma'] = df2['Total da Reserva']*0.15
df2.loc[(df2['Canal'] == 'Airbnb' )
       , 'Total da Reserva'] = df2['Ganhos'].apply(lambda x: x*1.0/0.85)
df2['Total da Reserva'] = df2['Total da Reserva'].round(2)
df2['Comissão Plataforma'] = df2['Comissão Plataforma'].round(2)

In [381]:
#criando valor por noite e ganho por noite
df2.loc[(df2['Canal'] == 'Airbnb' )
       , 'Valor por Noite'] = df2.apply(lambda row: row['Total da Reserva']/row['Número de Noites'],
                                        axis=1).round(2)
df2['Ganho por Noite'] = df2.apply(lambda row: row['Ganhos']/row['Número de Noites'], axis=1).round(2)
df2.loc[(df2['Ganhos'] == 0 )
       , 'Ganhos'] = df2['Preço de Venda Corrigido']

In [382]:
df2.dtypes

Nome Interno do Anúncio                object
Canal                                  object
Data de Criação                datetime64[ns]
Atendecência Reserva (dias)             int64
Chegada                        datetime64[ns]
Data de checkout               datetime64[ns]
Número de Noites                        int64
Adults Count                            int64
Children Count                          int64
Infants Count                           int64
Total de Hóspedes                       int64
Total da Reserva                      float64
Preço de Venda Corrigido              float64
Valor por Noite                       float64
Ganhos                                float64
Status da Reserva                      object
OTA's external code                    object
Tipo de Acomodação                      int64
Bairro                                 object
Comissão Plataforma                   float64
Ganho por Noite                       float64
dtype: object

In [402]:
#Resolvendo o valor int que veio na coluna objeto(str)
df2['Nome Interno do Anúncio'][550] = 'Apê Amplo Ipanema 300 m da Praia'
df2['Nome Interno do Anúncio'][533] = 'Apê Amplo Ipanema 300 m da Praia'
df2['Nome Interno do Anúncio'][529] = 'Apê Amplo Ipanema 300 m da Praia'

/var/folders/3y/b_kj5_xd25n0pc7gt9_qym6r0000gn/T/ipykernel_1535/2031407715.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Nome Interno do Anúncio'][550] = 'Apê Amplo Ipanema 300 m da Praia'
/var/folders/3y/b_kj5_xd25n0pc7gt9_qym6r0000gn/T/ipykernel_1535/2031407715.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Nome Interno do Anúncio'][533] = 'Apê Amplo Ipanema 300 m da Praia'
/var/folders/3y/b_kj5_xd25n0pc7gt9_qym6r0000gn/T/ipykernel_1535/2031407715.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.

In [384]:
#Descartando preço corrigido, pois era igual a coluna 'ganhos'
df2 = df2.drop ("Preço de Venda Corrigido", axis=1)

In [385]:
df2['Nome Interno do Anúncio'].unique()

array(['Studio Praia Ipanema', 'Charmoso Apartamento no Leblon!',
       'Apê Amplo Ipanema 300 m da Praia', 'Stúdio Praia Leblon',
       'Apê Amplo em Ipanema 300 m da Praia',
       'Ipanema Studio 300 m da praia', 'Super Apê com Jacuzzi Ipanema',
       'Apartamento Leblon Vista Mar!',
       'Copanema linda kitnet\n300 m da praia e Arpoador',
       'Apê Ipanema à 600m da praia', 'Ipanema à 200 m da praia',
       'Ipanema à 300 m da Praia',
       'Casa Cercada pela Natureza em Visconde de Mauá',
       'Ipanema à 350 m da Praia', 'Quarto solteiro Ipanema',
       'Suíte em Ipanema 300m da praia',
       '300 metros da praia em Ipanema.', 'Ipanema Amplo Vista Mar!',
       'Ipanema Amplo Vista Mar', 'Praia Stúdio Ipanema',
       'Charmoso Apartamento no Leblon', 501], dtype=object)

In [386]:
#Preenchendo tipo de acomodação
df2.loc[(df2['Nome Interno do Anúncio'].str.contains(pat = 'Ipanema à 200 m da praia', na = False))
       , 'Tipo de Acomodação'] = 'Quarto'
df2.loc[(df2['Nome Interno do Anúncio'].str.contains(pat = 'Ipanema à 300 m da Praia', na = False))
       , 'Tipo de Acomodação'] = 'Quarto'
df2.loc[(df2['Nome Interno do Anúncio'].str.contains(pat = 'Ipanema à 350 m da Praia', na = False))
       , 'Tipo de Acomodação'] = 'Quarto'
df2.loc[(df2['Nome Interno do Anúncio'].str.contains(pat = 'Suíte em Ipanema 300m da praia', na = False))
       , 'Tipo de Acomodação'] = 'Quarto'
df2.loc[(df2['Nome Interno do Anúncio'].str.contains(pat = '300 metros da praia em Ipanema.', na = False))
       , 'Tipo de Acomodação'] = 'Quarto'
df2.loc[(df2['Nome Interno do Anúncio'].str.contains(pat = 'Quarto solteiro Ipanema', na = False))
       , 'Tipo de Acomodação'] = 'Quarto'
df2.loc[(df2['Tipo de Acomodação'] == 0)
       , 'Tipo de Acomodação'] = 'Espaço Inteiro'

In [387]:
df2.head(4)

,Nome Interno do Anúncio,Canal,Data de Criação,Atendecência Reserva (dias),Chegada,Data de checkout,Número de Noites,Adults Count,Children Count,Infants Count,Total de Hóspedes,Total da Reserva,Valor por Noite,Ganhos,Status da Reserva,OTA's external code,Tipo de Acomodação,Bairro,Comissão Plataforma,Ganho por Noite
0,Studio Praia Ipanema,Airbnb,2023-08-07,0,2023-08-12,2023-08-15,3,2,0,0,2,971.39,323.80,825.68,Hóspede anterior,HMYM2TERN4,Espaço Inteiro,Ipanema,145.71,275.23
1,Charmoso Apartamento no Leblon!,Airbnb,2023-08-08,0,2023-08-11,2023-08-22,11,1,0,0,1,4902.52,445.68,4167.14,Hóspede anterior,HMK9A3HPXS,Espaço Inteiro,Leblon,735.38,378.83
2,Apê Amplo Ipanema 300 m da Praia,Airbnb,2023-07-11,0,2023-08-11,2023-08-14,3,8,0,0,8,2052.22,684.07,1744.39,Hóspede anterior,HMRSW4ZSKM,Espaço Inteiro,Ipanema,307.83,581.46
3,Stúdio Praia Leblon,Airbnb,2023-07-11,0,2023-08-11,2023-08-13,2,2,0,0,2,820.88,410.44,697.75,Hóspede anterior,HMXPZ8WSSP,Espaço Inteiro,Leblon,123.13,348.88


In [388]:
df2.columns

Index(['Nome Interno do Anúncio', 'Canal', 'Data de Criação',
       'Atendecência Reserva (dias)', 'Chegada', 'Data de checkout',
       'Número de Noites', 'Adults Count', 'Children Count', 'Infants Count',
       'Total de Hóspedes', 'Total da Reserva', 'Valor por Noite', 'Ganhos',
       'Status da Reserva', 'OTA's external code', 'Tipo de Acomodação',
       'Bairro', 'Comissão Plataforma', 'Ganho por Noite'],
      dtype='object')

In [389]:
# Preenchendo antecedência da reserva
df2['Atendecência Reserva (dias)'] = df2['Chegada'] - df2['Data de Criação']

In [390]:
# Criando novas colunas para análise
df2['Ganho por hospede'] = df2['Ganhos']/df2['Total de Hóspedes']
df2['Total da reserva por hospede'] = df2['Total da Reserva']/df2['Total de Hóspedes']
df2['Total da reserva por noite'] = df2['Total da Reserva']/df2['Número de Noites']
df2['Total da reserva por noite'] = (df2['Total da Reserva']/df2['Número de Noites']).round(2)

In [391]:
#arredondando 2 casas
df2['Ganho por hospede'] = df2['Ganho por hospede'].round(2)

In [392]:
df2['Status da Reserva'].unique()

array(['Hóspede anterior', 'Cancelada pelo Airbnb',
       'Cancelada pelo hóspede', 'Cancelada por você', 'Avaliar hóspede',
       'Aguardando avaliação do hóspede', 'Confirmada',
       'Estadia em andamento', 'Pedido de alteração da reserva',
       'reserva'], dtype=object)

In [393]:
# Preenchendo status da reserva
searchfor = ['Cancelada', 'cancelada']
df2.loc[(df2['Status da Reserva'].str.contains('|'.join(searchfor)))
       , 'Status da Reserva'] = 'Cancelada'
df2.loc[(~(df2['Status da Reserva'].str.contains('|'.join(searchfor))))
       , 'Status da Reserva'] = 'Confirmada'

In [423]:
df2.head(2)

,Nome Interno do Anúncio,Canal,Tipo de Acomodação,Porte,Data de Criação,Atendecência Reserva (dias),Chegada,Data de checkout,Número de Noites,Adults Count,Children Count,Infants Count,Total de Hóspedes,Total da Reserva,Total da reserva por hospede,Total da reserva por noite,Valor por Noite,Ganhos,Ganho por hospede,Ganho por Noite,Comissão Plataforma,Bairro,Status da Reserva,OTA's external code
0,Studio Praia Ipanema,Airbnb,Espaço Inteiro,Pequeno,2023-08-07,5,2023-08-12,2023-08-15,3,2,0,0,2,971.39,485.695,323.80,323.80,825.68,412.84,275.23,145.71,Ipanema,Confirmada,HMYM2TERN4
1,Charmoso Apartamento no Leblon!,Airbnb,Espaço Inteiro,Pequeno,2023-08-08,3,2023-08-11,2023-08-22,11,1,0,0,1,4902.52,4902.520,445.68,445.68,4167.14,4167.14,378.83,735.38,Leblon,Confirmada,HMK9A3HPXS


## Exploratory Analysis

In [396]:
df2['Atendecência Reserva (dias)'] = df2['Atendecência Reserva (dias)'].astype(str)
df2['Atendecência Reserva (dias)'] = df2['Atendecência Reserva (dias)'].map(lambda x: x.rstrip('days'))
df2['Atendecência Reserva (dias)'] = df2['Atendecência Reserva (dias)'].astype(int)

In [397]:
1 - separar dias, meses e ano
2 - separar apartamentos grandes de pequenos
3 - hipóteses: como se comporta a antecedência em relação ao valor da diária? 
qual foi a taxa de ocupação do por mês e por bairro? 
ranking de taxa de ocupação, receita por apartamento e por período
Como se comporta o valor da diária ao longo dos meses? 
comparar valor unitário da diária separando comparando com aptos peques e grandes
espaço inteito x quarto


SyntaxError: invalid syntax (2004615619.py, line 1)

In [399]:
df2_cancel = df2[df2['Status da Reserva'] == 'Cancelada']
df2 = df2[df2['Status da Reserva'] == 'Confirmada']

In [411]:
df2['Porte'] = 'Hello'
df2.loc[(df2['Nome Interno do Anúncio'] == 'Stúdio Praia Leblon')
       , 'Porte'] = 'Pequeno'
df2.loc[(df2['Nome Interno do Anúncio'] == 'Studio Praia Ipanema')
       , 'Porte'] = 'Pequeno'
df2.loc[(df2['Nome Interno do Anúncio'] == 'Charmoso Apartamento no Leblon!')
       , 'Porte'] = 'Pequeno'
df2.loc[(df2['Nome Interno do Anúncio'] == 'Apê Amplo Ipanema 300 m da Praia')
       , 'Porte'] = 'Grande'
df2.loc[(df2['Nome Interno do Anúncio'] ==  'Apê Amplo em Ipanema 300 m da Praia')
       , 'Porte'] = 'Grande'
df2.loc[(df2['Nome Interno do Anúncio'] == 'Ipanema Studio 300 m da praia')
       , 'Porte'] = 'Pequeno'
df2.loc[(df2['Nome Interno do Anúncio'] == 'Super Apê com Jacuzzi Ipanema')
       , 'Porte'] = 'Grande'
df2.loc[(df2['Nome Interno do Anúncio'] == 'Apartamento Leblon Vista Mar!')
       , 'Porte'] = 'Pequeno'
df2.loc[(df2['Nome Interno do Anúncio'] ==  'Copanema linda kitnet\n300 m da praia e Arpoador')
       , 'Porte'] = 'Pequeno'
df2.loc[(df2['Nome Interno do Anúncio'] == 'Apê Ipanema à 600m da praia')
       , 'Porte'] = 'Pequeno'
df2.loc[(df2['Nome Interno do Anúncio'] == 'Ipanema à 300 m da Praia')
       , 'Porte'] = 'Quarto'
df2.loc[(df2['Nome Interno do Anúncio'] == 'Casa Cercada pela Natureza em Visconde de Mauá')
       , 'Porte'] = 'Grande'
df2.loc[(df2['Nome Interno do Anúncio'] == 'Ipanema à 350 m da Praia')
       , 'Porte'] = 'Quarto'
df2.loc[(df2['Nome Interno do Anúncio'] == 'Ipanema à 350 m da Praia')
       , 'Porte'] = 'Quarto'
df2.loc[(df2['Nome Interno do Anúncio'] == 'Quarto solteiro Ipanema')
       , 'Porte'] = 'Quarto'
df2.loc[(df2['Nome Interno do Anúncio'] == 'Suíte em Ipanema 300m da praia')
       , 'Porte'] = 'Quarto'
df2.loc[(df2['Nome Interno do Anúncio'] == '300 metros da praia em Ipanema.')
       , 'Porte'] = 'Quarto'
df2.loc[(df2['Nome Interno do Anúncio'] == 'Ipanema Amplo Vista Mar!')
       , 'Porte'] = 'Grande'
df2.loc[(df2['Nome Interno do Anúncio'] ==  'Ipanema Amplo Vista Mar')
       , 'Porte'] = 'Grande'
df2.loc[(df2['Nome Interno do Anúncio'] == 'Praia Stúdio Ipanema')
       , 'Porte'] = 'Pequeno'
df2.loc[(df2['Nome Interno do Anúncio'] == 'Ipanema à 200 m da praia')
       , 'Porte'] = 'Quarto'
df2.loc[(df2['Nome Interno do Anúncio'] == 'Charmoso Apartamento no Leblon')
       , 'Porte'] = 'Quarto'

/var/folders/3y/b_kj5_xd25n0pc7gt9_qym6r0000gn/T/ipykernel_1535/3151867952.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Porte'] = 'Hello'


In [ ]:
#df2['booking day'] = df2['Data de Criação'].dt.day
#df2['booking month'] = df2['Data de Criação'].dt.month
#df2['booking year'] = df2['Data de Criação'].dt.year
#df2

In [425]:
#Arrumando colunas
df2 = df2[['Nome Interno do Anúncio', 'Canal' , 'Tipo de Acomodação', 'Porte', 'Atendecência Reserva (dias)', 'Data de Criação', 
           'Chegada', 
                 'Data de checkout','Número de Noites', 'Adults Count', 'Children Count', 'Infants Count',
                 'Total de Hóspedes', 'Total da Reserva', 'Total da reserva por hospede', 
          'Total da reserva por noite', 'Valor por Noite', 'Ganhos', 
          'Ganho por hospede', 'Ganho por Noite', 'Comissão Plataforma',  'Bairro',
          'Status da Reserva', "OTA's external code"]]

In [401]:
df2.dtypes

Nome Interno do Anúncio                 object
Canal                                   object
Tipo de Acomodação                      object
Data de Criação                 datetime64[ns]
Atendecência Reserva (dias)              int64
Chegada                         datetime64[ns]
Data de checkout                datetime64[ns]
Número de Noites                         int64
Adults Count                             int64
Children Count                           int64
Infants Count                            int64
Total de Hóspedes                        int64
Total da Reserva                       float64
Total da reserva por hospede           float64
Total da reserva por noite             float64
Valor por Noite                        float64
Ganhos                                 float64
Ganho por hospede                      float64
Ganho por Noite                        float64
Comissão Plataforma                    float64
Bairro                                  object
Status da Res

In [426]:
df2.to_csv('base_limpa_reservas.csv', sep='\t', encoding='utf-8')